In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox, scrolledtext
import requests
from docx import Document
import PyPDF2
import ttkbootstrap as tb

# ========== Ollama Configuration ==========
OLLAMA_URL = "http://localhost:11434/v1/chat/completions"

def check_ollama_running():
    try:
        response = requests.get("http://localhost:11434")
        return response.status_code == 200
    except requests.ConnectionError:
        return False

def ask_ollama(prompt):
    headers = {"Content-Type": "application/json"}
    data = {
        "model": "llama3",
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.7
    }
    response = requests.post(OLLAMA_URL, headers=headers, json=data)
    if response.status_code == 200:
        return response.json()['choices'][0]['message']['content']
    else:
        return f"❌ Error: {response.status_code}, {response.text}"

# ========== File Extraction ==========
def extract_text_from_docx(file_path):
    try:
        doc = Document(file_path)
        return '\n'.join([para.text for para in doc.paragraphs])
    except Exception as e:
        return f"❌ Error reading DOCX file: {e}"

def extract_text_from_pdf(file_path):
    try:
        with open(file_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            return '\n'.join([page.extract_text() for page in reader.pages if page.extract_text()])
    except Exception as e:
        return f"❌ Error reading PDF file: {e}"

# ========== CV Analysis ==========
def analyze_cv(cv_path, jd_path):
    if not check_ollama_running():
        return "❌ Ollama is not running. Please start it first using 'ollama serve'."

    # Extract resume text
    if cv_path.endswith(".docx"):
        cv_text = extract_text_from_docx(cv_path)
    elif cv_path.endswith(".pdf"):
        cv_text = extract_text_from_pdf(cv_path)
    else:
        return "❌ Unsupported CV file format."

    # Extract job description text
    if jd_path.endswith(".docx"):
        jd_text = extract_text_from_docx(jd_path)
    elif jd_path.endswith(".pdf"):
        jd_text = extract_text_from_pdf(jd_path)
    else:
        return "❌ Unsupported job description file format."

    if not cv_text or not jd_text:
        return "❌ Failed to extract text from one or both files."

    prompt = f"""
    Compare the following resume with the job description:
    Resume: {cv_text}

    Job Description: {jd_text}

    Provide a similarity score (0-100), highlight key matches, and suggest areas to improve in the resume.
    """
    return ask_ollama(prompt)

# ========== GUI Class ==========
class CVAnalyzerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("📄 CV Analyzer with Ollama")
        self.root.geometry("800x650")
        self.style = tb.Style("darkly")

        self.cv_path = tk.StringVar()
        self.jd_path = tk.StringVar()

        self.create_widgets()

    def create_widgets(self):
        padding = {'padx': 10, 'pady': 5}

        tb.Label(self.root, text="Select CV File (.docx/.pdf):", font=("Helvetica", 12)).pack(**padding)
        tb.Entry(self.root, textvariable=self.cv_path, width=80).pack()
        tb.Button(self.root, text="Browse CV", command=self.browse_cv, bootstyle="primary").pack(pady=5)

        tb.Label(self.root, text="Select Job Description File (.docx/.pdf):", font=("Helvetica", 12)).pack(**padding)
        tb.Entry(self.root, textvariable=self.jd_path, width=80).pack()
        tb.Button(self.root, text="Browse Job Description", command=self.browse_jd, bootstyle="primary").pack(pady=5)

        tb.Button(self.root, text="🔍 Analyze", bootstyle="success", command=self.run_analysis).pack(pady=20)

        self.output = scrolledtext.ScrolledText(self.root, wrap=tk.WORD, font=("Courier", 11), height=20)
        self.output.pack(fill=tk.BOTH, padx=10, pady=10, expand=True)

    def browse_cv(self):
        path = filedialog.askopenfilename(filetypes=[("Word or PDF", "*.docx *.pdf")])
        if path:
            self.cv_path.set(path)

    def browse_jd(self):
        path = filedialog.askopenfilename(filetypes=[("Word or PDF", "*.docx *.pdf")])
        if path:
            self.jd_path.set(path)

    def run_analysis(self):
        cv_file = self.cv_path.get()
        jd_file = self.jd_path.get()

        if not cv_file or not jd_file:
            messagebox.showwarning("Missing Files", "Please select both CV and Job Description files.")
            return

        self.output.delete("1.0", tk.END)
        self.output.insert(tk.END, "⏳ Analyzing... please wait...\n")
        self.root.update()

        result = analyze_cv(cv_file, jd_file)
        self.output.delete("1.0", tk.END)
        self.output.insert(tk.END, result)

# ========== Launch App ==========
if __name__ == "__main__":
    root = tb.Window(themename="darkly")
    app = CVAnalyzerApp(root)
    root.mainloop()
